# Integraçao dos dados

### Imports

In [67]:
import pandas as pd

## Carregar raw data

### Dados dos municípios

In [68]:
df_municipio = pd.read_csv('data/municipio/municipios_IBGE_ANSI.txt', sep=",", encoding='latin1')
df_municipio.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [69]:
df_populacao = pd.read_csv('data/municipio/populacaoMunicipiosEstimada2011_2021.csv', sep=";", skiprows=4, nrows=5597, encoding='latin1', header=None)
df_populacao.drop(columns=[4 ,6], inplace=True)
df_populacao.columns = ["tipo", "codigo_ibge", "nome", "populacao_2011", "populacao_2021"]
df_populacao.head()

,tipo,codigo_ibge,nome,populacao_2011,populacao_2021
0,UF,11,RondÃ´nia,1576455,1815278
1,MU,1100015,Alta Floresta D'Oeste,24228,22516
2,MU,1100379,Alto Alegre dos Parecis,12825,13268
3,MU,1100403,Alto ParaÃ­so,17444,22258
4,MU,1100346,Alvorada D'Oeste,16625,13807


In [70]:
df_area = pd.read_excel(open('data/municipio/AreaMunicipios_2021.xls', 'rb'), sheet_name='AR_BR_MUN_2021', nrows=5572) 
df_area.head()

,ID,CD_UF,NM_UF,NM_UF_SIGLA,CD_MUN,NM_MUN,AR_MUN_2021
0,1,11,Rondônia,RO,1100015,ALTA FLORESTA D'OESTE,7067.127
1,2,11,Rondônia,RO,1100023,ARIQUEMES,4426.571
2,3,11,Rondônia,RO,1100031,CABIXI,1314.352
3,4,11,Rondônia,RO,1100049,CACOAL,3793.000
4,5,11,Rondônia,RO,1100056,CEREJEIRAS,2783.300


## Merging dataframes para o banco

**Tabela Município**

Atributos:
* codigo_ibge (chave primária) ok
* codigo_uf (chave estrangeira) ok
* id_regiao_saude (chave estrangeira) ---> pendente pois o código está faltando o último dígito
* nome ok
* latitude ok
* longitude ok
* populacao_2011 ok
* populacao_2021 ok
* area ok
* densidade

In [71]:
tb_municipio = df_municipio[["codigo_ibge", "nome", "latitude", "longitude", "codigo_uf"]]
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31
2,5200100,Abadiânia,-16.19700,-48.7057,52
3,3100203,Abaeté,-19.15510,-45.4444,31
4,1500107,Abaetetuba,-1.72183,-48.8788,15


In [72]:
tb_municipio = pd.merge(tb_municipio,df_populacao[['codigo_ibge','populacao_2011', 'populacao_2021']],on='codigo_ibge', how='left')
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439


In [73]:
tb_municipio = tb_municipio.merge(df_area[['CD_MUN','AR_MUN_2021']], how='left', left_on='codigo_ibge', right_on='CD_MUN').drop(columns='CD_MUN')
tb_municipio.rename(columns={"AR_MUN_2021": "area_2021"}, inplace=True)
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021,area_2021
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158,143.357
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022,880.461
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873,1044.555
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263,1817.067
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439,1610.654


In [75]:
tb_municipio['densidade_2021']= tb_municipio['populacao_2021']/tb_municipio['area_2021']
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021,area_2021,densidade_2021
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158,143.357,63.882475
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022,880.461,7.975367
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873,1044.555,19.982672
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263,1817.067,12.802500
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439,1610.654,99.611090


In [77]:
tb_municipio[tb_municipio.nome=='Espumoso']

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021,area_2021,densidade_2021
1708,4307500,Espumoso,-28.7286,-52.8461,43,15241,15594,783.642,19.899393
